## Imports

In [9]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import os.path
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import cv2
import os
import sys
from os.path import exists
from scipy import spatial

## Settings

In [10]:
im_dir = '/home/drevital/obstacles_classification_datasets/rgb_6_balanced/train/no_obstacle'
imnames = os.listdir(im_dir)

## Load the MobileNet Module

In [11]:
module_handle = 'https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4'
module = hub.load(module_handle)

## compare image's ref, current parts

In [12]:
def compare_image(im_dir, im_name):
    im = tf.io.read_file(os.path.join(im_dir, im_name))
    im = tf.io.decode_jpeg(im, channels=3)
    im_src = im 
    w = im.shape[1]
    im1 = im[:, :w//2, :]
    im2 = im[:, w//2:, :]
    src_im1 = im1
    src_im2 = im2
    im1 = tf.image.resize_with_pad(im1, 224, 224)
    im1  = tf.image.convert_image_dtype(im1, tf.float32)[tf.newaxis, ...]
    im2 = tf.image.resize_with_pad(im2, 224, 224)
    im2  = tf.image.convert_image_dtype(im2, tf.float32)[tf.newaxis, ...]  
    f1 = module(im1)
    f1_set = np.squeeze(f1)
    f2 = module(im2)   
    f2_set = np.squeeze(f2)
    diff = spatial.distance.cosine(f1_set, f2_set)
    
    return diff

In [13]:
diffs = []

for im_name in tqdm(imnames):
    diffs.append(compare_image(im_dir, im_name))

100%|██████████| 9643/9643 [14:26<00:00, 11.12it/s]


In [16]:
np.mean(diffs)

0.11951494198684609

In [17]:
np.std(diffs)

0.07264464152956103

In [18]:
np.max(diffs)

0.5973824858665466

In [14]:
#np.mean(diffs)
#0.21585878213344536

In [15]:
#np.std(diffs)
#0.09474018506618465